# 用 (txyz+qwxyz) 7列 pose.csv 生成 syntheticRT.txt 12列 (历史遗留格式)
//2016-10-28 21:30:45


### 关联: 
1. compareEulerAngle.ipynb //用于对比各种算法vs.GTruth
2. zsens-data-viewer.ipynb //之前做imu初值时一些代码, 抄过来一些

In [1]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [1]:
'拷贝自 depth-imu-calib-gen-synthetic.ipynb'

%pylab
import numbers
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO
from primesense import openni2

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix #r也用a表示
tf.r2m = tf.rotation_matrix
tf.m2a = tf.m2r
tf.a2m = tf.r2m
tf.a2q = tf.quaternion_about_axis
tf.qmul = tf.quaternion_multiply
tf.qinv = tf.quaternion_inverse

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

def q2a(qx, qy, qz, qw, vecLen = 4):
    '''
    参见 tf.quaternion_about_axis 实现;
    算法参见:
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToQuaternion/
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/
    
    测试代码：
    q=tf.a2q(0.123, (1,0,0))
    a=q2a(*q) #qxyzw顺序, 刻意与 tf 规则一致
    print q #[ 0.06146124  0.          0.          0.99810947]
    print a #(0.12300000000000065, 0.99999999999998979, 0.0, 0.0)
    '''
    if not ( isinstance(qx, numbers.Real) and isinstance(qy, numbers.Real) and isinstance(qz, numbers.Real) and isinstance(qw, numbers.Real) ):
        raise TypeError, 'Q:xyzw must be all numbers, not arrays / lists.'
        return
#     if not vecLen in (3, 4):
#         print 'vecLen 必须是 3 或者 4'
#         return
        
    angle = 2 * np.arccos(qw)
    'sin(angle/2):'
    sinA2 = np.sqrt(1 - qw * qw)
    x = qx / sinA2
    y = qy / sinA2
    z = qz / sinA2
    
    if vecLen == 4:
        return angle, x, y, z
    elif vecLen == 3:
        return x*angle, y*angle, z*angle
    else:
        raise ValueError, 'vecLen must be integer 3 or 4'
    pass

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from mpl_toolkits.mplot3d import axes3d

def plotAxAnim(qxyzws):
    '''
    关键帧mat矩阵不好看，改为3D绘制坐标轴，动画演示
    拷贝自 depth-imu-calib-gen-synthetic.ipynb, 改为函数, 2016-7-7 17:26:37
    qxyzws 必须是 local->global
    '''
    fig = figure()
    ax = fig.add_subplot(111, projection='3d')

    '添加xyz参考轴'
    o3d=(0,0,0)
    oxyz=np.eye(3)
    ax.plot_wireframe(*zip(o3d, oxyz[0]), color='r')
    ax.plot_wireframe(*zip(o3d, oxyz[1]), color='g')
    ax.plot_wireframe(*zip(o3d, oxyz[2]), color='b')

    wfx = wfy = wfz = None
    for i, q in enumerate(qxyzws):
    #     break
    #     print i
        oldwfx, oldwfy, oldwfz = wfx, wfy, wfz
    #     attribs = dat.attrib
    #     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
    #     rmat = tf.q2m(rxyzw)[:3, :3]
        rmat = tf.q2m(q)[:3, :3]

        '调试看mat'
    #     if i==108:
    #         print i, rmat
    #     elif i==136:
    #         print i, rmat

    #     print(rmat)
        vx = rmat[:,0]
        vy = rmat[:,1]
        vz = rmat[:,2]
        wfx = ax.plot_wireframe(*zip((0,0,0), vx), color='r')
        wfy = ax.plot_wireframe(*zip((0,0,0), vy), color='g')
        wfz = ax.plot_wireframe(*zip((0,0,0), vz), color='b')
        if oldwfx != None:
            ax.collections.remove(oldwfx)
            ax.collections.remove(oldwfy)
            ax.collections.remove(oldwfz)
            pass

    #     fig.suptitle(str(i), fontsize=20)
    #     tit = ax.set_title(str(i))
        ax.title.set_text('idx: %s'%i)
        plt.pause(0.01)
        pass
    pass


In [3]:
'''
7列->12列, 存文件 syntheticRT.txt
'''

# fn=r'E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf-png\zcpose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\syn_pose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep7\syn_pose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\syn_pose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\syn_pose.csv'
fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\syn_pose.csv'

fn=r'E:\sdf2sdf-data\Kinect_Bunny_Handheld\markerboard7n.csv'

# fn=r'E:\Github\pcl\zctest\_build.vc10\tmp\syn_pose.csv'

# fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\pose-ahc.csv'
# fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc.csv'

# fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-graph-opt-trunc0.csv'

dat = loadtxt(fn, delimiter=',')
print 'dat.shape:', dat.shape

tvecs = dat[:,:3]
qwxyz = dat[:,3:]
qxyzw = dat[:,[4,5,6,3]]
# figure()
# plot(qwxyz)
# legend(list('wxyz'))
# plot(qxyzw)
# ylim(ymax=1, ymin=-0.8)
# legend(list('xyzw'))
# axis([0,155,-0.4,1])
# axis([0,300,-0.4,1])

'如果也要画 tvecs'
# plot(tvecs)

rmats = asarray([tf.q2m(qi)[:3,:3].flatten() for qi in qxyzw])
print rmats.shape
'fake伪造 t向量'
tvecsFake = zeros((rmats.shape[0], 3))
# synMat = hstack((tvecsFake, rmats)) #输出矩阵
synMat = hstack((tvecs, rmats)) #输出矩阵, 单位米(m)
# synMat = hstack((tvecs*1e3, rmats)) #输出矩阵, t单位毫米(mm)

fnPath, _ = os.path.split(fn)
fnout = os.path.join(fnPath, 'syntheticRT.txt')
print 'fnout:', fnout
savetxt(fnout, synMat, delimiter=' ', fmt='%.8f')


dat.shape: (233L, 7L)
(233L, 9L)
fnout: E:\sdf2sdf-data\Kinect_Bunny_Handheld\syntheticRT.txt


In [42]:
'单单绘制 m1fn(一般是GTruth), 观察tvec //拷贝自compareEulerAngle.ipynb'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep0\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep1\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep2\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep3\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep4\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\syn_pose.csv'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep7\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\syn_pose.csv'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\syn_pose.csv'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\syn_pose.csv'

# m1fn=r'E:\Github\pcl\zctest\_build.vc10\syn_pose.csv'
# m1fn=r'E:\Github\pcl\_build.vc10\bin\zcpose.csv'
rng=slice(None)

# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-graph-opt-trunc0.csv'
# m1fn=r'E:\Github\pcl\_build.vc10\bin\zcpose-ahc-graph-opt.csv'
# rng = slice(285)

m1 = loadtxt(m1fn, delimiter=',')
# m1tvecs = m1[:, :3]
m1tvecs = m1[rng, :3]
close()
figure(figsize=(6,6))
axx,axy=0,1 #选用哪两个轴绘制
# pts1 = scatter(m1tvecs[:, 0], m1tvecs[:, 1], c='r')
pts1 = plot(m1tvecs[:, axx], m1tvecs[:, axy], c='r', ls='-')
annotate('start', m1tvecs[0, (axx,axy)])
annotate('end', m1tvecs[-1, (axx,axy)])
savefig('foo2.png')


In [43]:
'相机轨迹周长 @2017-7-9 15:07:57'
diffs=m1tvecs[1:]-m1tvecs[:-1]
sum(norm(diffs[:], axis=1))

24.220929012670187

In [12]:
'''
将 stanfordPCL-kinfuLS 输出的 8列: ts+txyz+qxyzw; 整理成之前 7列 txyz+qwxyz (w前置) 形式
@2017-7-17 23:22:06
'''

m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-t25-vs1.txt'
m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.txt'
m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-t25-vs1.txt'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\liv.klg-conf0.freiburg'

# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-bdr-t25-vs1.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-kf-t25-vs1.txt'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\liv.klg-conf0.freiburg'

# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-t25-vs1.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.txt'
# m1fn = r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-kf-t25-vs1.txt'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\liv.klg-conf0.freiburg'

# m1fn=r'E:\geo-cu399-pen\turntable\flycat1c-raw_frames-ts\cu&bdr\kinfu_poses-bdr-t5-vs1.txt'
m1fn=r'E:\geo-cu399-pen\turntable\bunny1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.txt'
m1fn=r'E:\geo-cu399-pen\turntable\budda1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.txt'

'+++++++++++++++eval NICP @2017-9-19 09:40:55'
m1fn=r'E:\geo-cu399-pen\data-archive\01-armadillo\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\02-dragon\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\03-bunny\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\04-lambunny\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\05-owl\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\06-wingedcat\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\07-buddahead\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\08-tooth\nicp_poses.txt'
m1fn=r'E:\geo-cu399-pen\data-archive\09-mug\nicp_poses.txt'

fold, fn = os.path.split(m1fn)
basefn, ext = os.path.splitext(fn)
fnOut = os.path.join(fold, basefn + '.csv')
m1=loadtxt(m1fn) #空格分割, col=8
m1out=zeros((m1.shape[0], 7))
m1out[:,:3]=m1[:,1:4]
m1out[:,3]=m1[:,-1]
m1out[:,4:]=m1[:,4:7]
savetxt(fnOut, m1out, fmt='%.9f', delimiter=',')

fnOut

'E:\\geo-cu399-pen\\data-archive\\09-mug\\nicp_poses.csv'